In [ ]:
pip install yfinance

In [1]:
from helpers import *
import yfinance as yf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm
from datetime import date, timedelta, datetime

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
from options_helpers import *
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import CubicSpline
from KalshiClientsBaseV2 import ExchangeClient

In [3]:
prod_email, prod_password = get_kalshi_creds()
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2" 
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

In [4]:
annualized_rate = .05374
ir = np.log(1+annualized_rate)

In [5]:
ndx_df, spx_df = get_ndx_spx_data()
    
ndx_df = ndx_df[ndx_df['Date'] >= date(2024, 1, 1)][['Date', 'Open', 'Close']].reset_index(drop=True)
spx_df = spx_df[spx_df['Date'] >= date(2024, 1, 1)][['Date', 'Open', 'Close']].reset_index(drop=True) #get daily index data for 2024 onwards


In [18]:
datetime.fromtimestamp(1711461600000/1000)

datetime.datetime(2024, 3, 26, 10, 0)

In [22]:
'''
gets daily opening prices for SPX and NDX options from some specified date onwards from Polygon and stores data in df
'''
import requests
params_ = {'adjusted': 'true', 'sort': 'asc', 'apiKey' : 'vGQU0FoV2y68f1aA4hFlncpgRNiJVroO'}
d = {'underlying': [], 'strike': [], 'type': [], 'date': [], 'price': []}


exception_rate = []

# -- NDX below --

for idx, row in ndx_df.iterrows(): #loop thru all days' worth of ohlcv option chain data for both spx and ndx including current date
    cur_dt = row['Date']
    year = str(cur_dt.year - 2000)
    month = cur_dt.month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    day = cur_dt.day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
        
    open_ = row['Open']
    close_ = row['Close']
    #get OTM calls and puts data
    lb_call_strike = open_ + 10 - open_ % 10
    ub_put_strike = open_ - open_ % 10
    call_strikes = np.arange(lb_call_strike, lb_call_strike + 500, 10)
    put_strikes = np.arange(ub_put_strike - 500, ub_put_strike + 10, 10)
    strikes = list(call_strikes) + list(put_strikes)
    day_data_ret_err = 0
    for sp in strikes:

        put_ticker = 'NDXP' + year + month + day + 'P' +  str(int(sp)) + '000'
        call_ticker = 'NDXP'+ year + month + day + 'C' + str(int(sp)) + '000'
        put_url = 'https://api.polygon.io/v1/open-close/O:{}/{}'.format(put_ticker, cur_dt.isoformat())
        call_url = 'https://api.polygon.io/v1/open-close/O:{}/{}'.format(call_ticker, cur_dt.isoformat())
        
        r = requests.get(put_url, params=params_).json()
        try:
            option_open = r['open']
            d['strike'].append(int(sp))
            d['underlying'].append('NDX')
            d['type'].append('put')
            d['date'].append(cur_dt)
            d['price'].append(option_open)
        except:
            day_data_ret_err += 1
            
        r = requests.get(call_url, params=params_).json()
        try:
            option_open = r['open']
            d['strike'].append(int(sp))
            d['underlying'].append('NDX')
            d['type'].append('call')
            d['date'].append(cur_dt)
            d['price'].append(option_open)
        except:
            day_data_ret_err += 1
        
        

# -- SPX below --

for idx, row in spx_df.iterrows():
    cur_dt = row['Date']
    year = str(cur_dt.year - 2000)
    month = cur_dt.month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    day = cur_dt.day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
    
    open_ = row['Open']
    #get OTM calls and puts data
    lb_call_strike = open_ + 5 - open_ % 5
    ub_put_strike = open_ - open_ % 5
    call_strikes = np.arange(lb_call_strike, lb_call_strike + 155, 5)
    put_strikes = np.arange(ub_put_strike - 150, ub_put_strike + 5, 5)
    strikes = list(call_strikes) + list(put_strikes)
    day_data_ret_err = 0
    for sp in strikes:
        put_ticker = 'SPXW' + year + month + day + 'P' + '0' + str(int(sp)) + '000'
        call_ticker = 'SPXW'+ year + month + day + 'C' + '0' + str(int(sp)) + '000'
        put_url = 'https://api.polygon.io/v1/open-close/O:{}/{}'.format(put_ticker, cur_dt.isoformat())
        call_url = 'https://api.polygon.io/v1/open-close/O:{}/{}'.format(call_ticker, cur_dt.isoformat())
        
        r = requests.get(put_url, params=params_).json()
        try:
            option_open = r['open']
            d['strike'].append(int(sp))
            d['underlying'].append('SPX')
            d['type'].append('put')
            d['date'].append(cur_dt)
            d['price'].append(option_open)
        except:
            day_data_ret_err += 1
            
        r = requests.get(call_url, params=params_).json()
        try:
            option_open = r['open']
            d['strike'].append(int(sp))
            d['underlying'].append('SPX')
            d['type'].append('call')
            d['date'].append(cur_dt)
            d['price'].append(option_open)
        except:
            day_data_ret_err += 1
        
        

df = pd.DataFrame(data=d) #creates dataframe of strike-underlying-expiration date-type-opening price columns

ndx_opt_data = df[df['underlying'] == 'NDX']
spx_opt_data = df[df['underlying'] == 'SPX']
ndx_df['date'] = ndx_df.apply(lambda x: x['Date'], axis=1)
spx_df['date'] = spx_df.apply(lambda x: x['Date'], axis=1)

ndx_df = ndx_df[['date', 'Open', 'Close']]
spx_df = spx_df[['date', 'Open', 'Close']]

ndx_opt_data = ndx_opt_data.merge(ndx_df, how='inner', on='date')

ndx_itm_data = ndx_opt_data[((ndx_opt_data['strike'] > ndx_opt_data['Open']) & (ndx_opt_data['type']=='put')) | ((ndx_opt_data['strike'] < ndx_opt_data['Open']) & (ndx_opt_data['type']=='call'))]
ndx_otm_data = ndx_opt_data[((ndx_opt_data['strike'] > ndx_opt_data['Open']) & (ndx_opt_data['type']=='call')) | ((ndx_opt_data['strike'] < ndx_opt_data['Open']) & (ndx_opt_data['type']=='put'))]

spx_opt_data = spx_opt_data.merge(spx_df, how='inner', on='date')
spx_itm_data = spx_opt_data[((spx_opt_data['strike'] > spx_opt_data['Open']) & (spx_opt_data['type']=='put')) | ((spx_opt_data['strike'] < spx_opt_data['Open']) & (spx_opt_data['type']=='call'))]
spx_otm_data = spx_opt_data[((spx_opt_data['strike'] > spx_opt_data['Open']) & (spx_opt_data['type']=='call')) | ((spx_opt_data['strike'] < spx_opt_data['Open']) & (spx_opt_data['type']=='put'))]


df_itm = pd.concat([ndx_itm_data, spx_itm_data])
df_otm = pd.concat([ndx_otm_data, spx_otm_data])


df_itm = df_itm.sort_values(by=['date', 'strike']).reset_index(drop=True)
df_otm = df_otm.sort_values(by=['date', 'strike']).reset_index(drop=True)


In [23]:
df_otm

,underlying,strike,type,date,price,Open,Close
0,SPX,4595,put,2024-01-02,0.05,4745.200195,4742.830078
1,SPX,4600,put,2024-01-02,0.05,4745.200195,4742.830078
2,SPX,4605,put,2024-01-02,0.05,4745.200195,4742.830078
3,SPX,4610,put,2024-01-02,0.05,4745.200195,4742.830078
4,SPX,4615,put,2024-01-02,0.05,4745.200195,4742.830078
...,...,...,...,...,...,...,...
8704,NDX,18710,call,2024-03-26,0.20,18329.300781,18210.537109
8705,NDX,18720,call,2024-03-26,0.30,18329.300781,18210.537109
8706,NDX,18750,call,2024-03-26,0.15,18329.300781,18210.537109
8707,NDX,18780,call,2024-03-26,0.28,18329.300781,18210.537109


In [25]:
df_otm[(df_otm['date']==date(2024, 3, 26)) & (df_otm['underlying']=='SPX')]

,underlying,strike,type,date,price,Open,Close
8556,SPX,5080,put,2024-03-26,0.05,5228.850098,5203.580078
8557,SPX,5085,put,2024-03-26,0.05,5228.850098,5203.580078
8558,SPX,5090,put,2024-03-26,0.05,5228.850098,5203.580078
8559,SPX,5095,put,2024-03-26,0.05,5228.850098,5203.580078
8560,SPX,5100,put,2024-03-26,0.05,5228.850098,5203.580078
8561,SPX,5105,put,2024-03-26,0.05,5228.850098,5203.580078
8562,SPX,5110,put,2024-03-26,0.05,5228.850098,5203.580078
8563,SPX,5115,put,2024-03-26,0.05,5228.850098,5203.580078
8564,SPX,5120,put,2024-03-26,0.05,5228.850098,5203.580078
8565,SPX,5125,put,2024-03-26,0.05,5228.850098,5203.580078


In [26]:
df_otm['est_iv'] = df_otm.apply(lambda x: implied_volatility_call(x['price'], x['Open'], x['strike'], 1/252, ir) if x['type']=='call' else implied_volatility_put(x['price'], x['Open'], x['strike'], 1/252, ir) , axis=1)

found on 5th iteration
difference is equal to 5.314058554510559e-05
found on 5th iteration
difference is equal to 9.06392727422628e-05
found on 6th iteration
difference is equal to 1.5865620887900445e-07
found on 6th iteration
difference is equal to 3.723913007264157e-07
found on 6th iteration
difference is equal to 7.998692026917054e-07
found on 6th iteration
difference is equal to 1.58717438196454e-06
found on 6th iteration
difference is equal to 2.9332584759317326e-06
found on 6th iteration
difference is equal to 6.74115970433764e-07
found on 6th iteration
difference is equal to 8.31539601690856e-06
found on 6th iteration
difference is equal to 1.2898021368099033e-05
found on 6th iteration
difference is equal to 1.905640654768781e-05
found on 6th iteration
difference is equal to 2.6916150610387102e-05
found on 6th iteration
difference is equal to 3.6452369802472684e-05
found on 6th iteration
difference is equal to 1.0732648924538513e-05
found on 6th iteration
difference is equal to 

found on 5th iteration
difference is equal to 9.523937638533653e-09
found on 5th iteration
difference is equal to 2.882427452277625e-08
found on 5th iteration
difference is equal to 9.98811242602926e-07
found on 6th iteration
difference is equal to 3.6037590689375065e-09
found on 5th iteration
difference is equal to 1.6246211544945766e-06
found on 5th iteration
difference is equal to 8.522649181941233e-06
found on 6th iteration
difference is equal to 3.7713814438067317e-07
found on 6th iteration
difference is equal to 1.0294536525456977e-06
found on 5th iteration
difference is equal to 3.2678013644815707e-05
found on 6th iteration
difference is equal to 3.1255792809492533e-07
found on 5th iteration
difference is equal to 5.461866322775677e-05
found on 7th iteration
difference is equal to 3.4407736382302545e-08
found on 6th iteration
difference is equal to 3.6725757259992164e-05
found on 7th iteration
difference is equal to 3.911692823516111e-06
found on 6th iteration
difference is equa

found on 3th iteration
difference is equal to 1.7176296296383953e-05
found on 3th iteration
difference is equal to 7.019568112553998e-06
found on 3th iteration
difference is equal to 2.4100136215565726e-06
found on 3th iteration
difference is equal to 2.4519067665096372e-08
found on 3th iteration
difference is equal to 2.3346728994511068e-09
found on 3th iteration
difference is equal to 1.43337786084885e-10
found on 2th iteration
difference is equal to 5.718539978261106e-05
found on 2th iteration
difference is equal to 4.971557245880831e-06
found on 2th iteration
difference is equal to 6.562640919582918e-08
found on 2th iteration
difference is equal to 3.893774191965349e-12
found on 2th iteration
difference is equal to 1.4314700536033342e-07
found on 2th iteration
difference is equal to 3.0653020104409734e-06
found on 2th iteration
difference is equal to 3.783929322054291e-05
found on 3th iteration
difference is equal to 2.5995170460646477e-09
found on 3th iteration
difference is equal

found on 6th iteration
difference is equal to 1.0103736786948314e-06
found on 6th iteration
difference is equal to 5.73013906518538e-06
found on 6th iteration
difference is equal to 8.65045552983501e-06
found on 6th iteration
difference is equal to 8.381224717024516e-07
found on 7th iteration
difference is equal to 1.9624184667318967e-05
found on 6th iteration
difference is equal to 2.6670569207021266e-06
found on 6th iteration
difference is equal to 6.096888660600541e-09
found on 6th iteration
difference is equal to 9.499330962947639e-07
found on 6th iteration
difference is equal to 2.946316886021094e-07
found on 6th iteration
difference is equal to 5.3737867378167437e-08
found on 6th iteration
difference is equal to 7.42942097842203e-08
found on 6th iteration
difference is equal to 2.9794550715500634e-08
found on 6th iteration
difference is equal to 2.7670566623561044e-06
found on 5th iteration
difference is equal to 6.363916167861916e-05
found on 5th iteration
difference is equal to

found on 6th iteration
difference is equal to 6.784976576285073e-05
found on 6th iteration
difference is equal to 3.009838151213362e-06
found on 6th iteration
difference is equal to 1.2270375157052982e-06
found on 6th iteration
difference is equal to 6.517268479888649e-07
found on 5th iteration
difference is equal to 4.290723373057137e-05
found on 5th iteration
difference is equal to 9.613055044654928e-06
found on 5th iteration
difference is equal to 3.7064843581902096e-05
found on 6th iteration
difference is equal to 1.6326168861269252e-08
found on 5th iteration
difference is equal to 8.11503645808953e-05
found on 6th iteration
difference is equal to 5.1729091454788545e-08
found on 5th iteration
difference is equal to 2.000865187712364e-05
found on 5th iteration
difference is equal to 9.737270977527857e-06
found on 5th iteration
difference is equal to 1.0613101020240379e-07
found on 5th iteration
difference is equal to 1.442796948580849e-08
found on 5th iteration
difference is equal t

...

found on 5th iteration
difference is equal to 1.5863731162024663e-07
found on 5th iteration
difference is equal to 9.132179457083112e-09
found on 5th iteration
difference is equal to 1.7846502942120424e-08
found on 5th iteration
difference is equal to 1.6860537854768154e-08
found on 5th iteration
difference is equal to 5.375636602522604e-09
found on 4th iteration
difference is equal to 2.7505043369568938e-05
found on 5th iteration
difference is equal to 6.440291189235836e-09
found on 4th iteration
difference is equal to 3.107677762281824e-05
found on 4th iteration
difference is equal to 6.874904588727304e-06
found on 4th iteration
difference is equal to 3.4815927501874455e-05
found on 4th iteration
difference is equal to 1.2030430684539795e-05
found on 4th iteration
difference is equal to 5.809005365176034e-06
found on 4th iteration
difference is equal to 1.0348390469427926e-05
found on 4th iteration
difference is equal to 1.400719053901156e-05
found on 4th iteration
difference is equa

found on 6th iteration
difference is equal to 9.98204610311304e-06
found on 6th iteration
difference is equal to 9.388366409263105e-06
found on 6th iteration
difference is equal to 8.447079865442353e-06
found on 7th iteration
difference is equal to 1.3711480022216005e-07
found on 7th iteration
difference is equal to 1.0826967127008924e-07
found on 7th iteration
difference is equal to 8.11946783557671e-08
found on 6th iteration
difference is equal to 8.902882780929822e-05
found on 6th iteration
difference is equal to 7.31009895503093e-05
found on 6th iteration
difference is equal to 5.844605908293665e-05
found on 6th iteration
difference is equal to 4.5455794773371916e-05
found on 6th iteration
difference is equal to 3.434398584971354e-05
found on 6th iteration
difference is equal to 2.516683745028281e-05
found on 6th iteration
difference is equal to 1.7851437758051314e-05
found on 6th iteration
difference is equal to 8.069156731946303e-06
found on 6th iteration
difference is equal to 3

found on 5th iteration
difference is equal to 4.3997135890094796e-07
found on 5th iteration
difference is equal to 5.061493777702708e-05
found on 6th iteration
difference is equal to 4.5268260465136656e-07
found on 6th iteration
difference is equal to 6.205441906159681e-07
found on 6th iteration
difference is equal to 7.593897265734117e-07
found on 6th iteration
difference is equal to 9.712565210401314e-06
found on 6th iteration
difference is equal to 9.998329440458198e-06
found on 7th iteration
difference is equal to 1.9559135182312115e-07
found on 6th iteration
difference is equal to 9.086816697839728e-06
found on 6th iteration
difference is equal to 8.095838468585859e-06
found on 7th iteration
difference is equal to 1.2832323008360458e-07
found on 7th iteration
difference is equal to 1.0006424204511699e-07
found on 7th iteration
difference is equal to 7.409270778857557e-08
found on 6th iteration
difference is equal to 8.446573387459189e-05
found on 6th iteration
difference is equal 

found on 6th iteration
difference is equal to 4.293177117628555e-06
found on 6th iteration
difference is equal to 7.044794309504776e-06
found on 6th iteration
difference is equal to 1.1024792372180126e-05
found on 6th iteration
difference is equal to 1.6519441165027682e-05
found on 6th iteration
difference is equal to 2.378055780098698e-05
found on 6th iteration
difference is equal to 3.298517516441002e-05
found on 6th iteration
difference is equal to 4.419355996247709e-05
found on 6th iteration
difference is equal to 5.731018668768406e-05
found on 6th iteration
difference is equal to 7.205235567368795e-05
found on 6th iteration
difference is equal to 8.793080494910732e-05
found on 7th iteration
difference is equal to 7.882557610805474e-08
found on 7th iteration
difference is equal to 1.0425917622292058e-07
found on 7th iteration
difference is equal to 1.3015968462759542e-07
found on 6th iteration
difference is equal to 8.040705191086905e-06
found on 6th iteration
difference is equal t

found on 6th iteration
difference is equal to 4.853798807885612e-05
found on 6th iteration
difference is equal to 9.819616822420718e-06
found on 6th iteration
difference is equal to 4.9991878690314184e-05
found on 7th iteration
difference is equal to 1.8053504717563396e-07
found on 7th iteration
difference is equal to 1.5870555784835982e-07
found on 7th iteration
difference is equal to 1.319759029144718e-07
found on 7th iteration
difference is equal to 1.0405004573221799e-07
found on 7th iteration
difference is equal to 7.785914206415168e-08
found on 6th iteration
difference is equal to 8.706519957328573e-05
found on 6th iteration
difference is equal to 7.136388927690696e-05
found on 6th iteration
difference is equal to 5.6938229651845584e-05
found on 6th iteration
difference is equal to 4.41747401509901e-05
found on 6th iteration
difference is equal to 3.328138795026525e-05
found on 6th iteration
difference is equal to 2.430858703376948e-05
found on 6th iteration
difference is equal t

c = S_0 N(d1) - K * e^(-rT) * N(d2)

c = e^ (-rT) * E [max(S-K, 0)]
 = e ^ (-rT) * sum (p(Z) * (Z-k)) for Z > k
 = e ^ (-rT) * p (S > k) * E[S-K| S > K]
 = e ^ (-rT) * p(S > K) * (E[S| S > K] - K)
 = e ^ (-rT) * p(S > K) * (E[S| S > K]) - K * e ^ (-rT) * p(S > K)
 
 so p(S>K) ~=~ N(d2)
 where d2 = (ln(S_0/K) + (r-(sigma**2)/2) * T)/(sigma * sqrt(T))
 

In [27]:
def calc_geq_strike_prob(S_0, K, r, sigma, T):
    d2 = (np.log(S_0/K) + (r - (sigma**2)/2)*T)/(sigma * np.sqrt(T))
    return norm.cdf(d2)

In [28]:
df_otm['geq_strike_prob'] = df_otm.apply(lambda x:calc_geq_strike_prob(x['Open'], x['strike'], ir, x['est_iv'], 1/252), axis=1)

In [29]:
df_otm[(df_otm['underlying']=='SPX') & (df_otm['strike'] > 5200) & (df_otm['strike'] < 5300) & (df_otm['date'] == date(2024,3,26))]

,underlying,strike,type,date,price,Open,Close,est_iv,geq_strike_prob
8581,SPX,5205,put,2024-03-26,1.25,5228.850098,5203.580078,0.064530,0.879746
8582,SPX,5210,put,2024-03-26,1.80,5228.850098,5203.580078,0.062356,0.834060
8583,SPX,5215,put,2024-03-26,2.55,5228.850098,5203.580078,0.059911,0.775152
8584,SPX,5220,put,2024-03-26,3.70,5228.850098,5203.580078,0.058384,0.696804
8585,SPX,5225,put,2024-03-26,5.28,5228.850098,5203.580078,0.057029,0.602978
8586,SPX,5230,call,2024-03-26,8.40,5228.850098,5203.580078,0.064165,0.497993
8587,SPX,5235,call,2024-03-26,5.81,5228.850098,5203.580078,0.061540,0.400686
8588,SPX,5240,call,2024-03-26,4.00,5228.850098,5203.580078,0.061204,0.308347
8589,SPX,5245,call,2024-03-26,2.50,5228.850098,5203.580078,0.059520,0.220958
8590,SPX,5250,call,2024-03-26,1.60,5228.850098,5203.580078,0.059884,0.154600


In [35]:
exchange_client.get_event('INXU-24MAR26')['markets']

[{'ticker': 'INXU-24MAR26-T5049.99',
  'event_ticker': '',
  'market_type': 'binary',
  'title': '',
  'subtitle': '5,050 or above',
  'yes_sub_title': '5,050 or above',
  'no_sub_title': 'Outside 5,050 or above',
  'open_time': '2024-03-25T20:00:00Z',
  'close_time': '2024-03-26T20:00:00Z',
  'expected_expiration_time': '2024-03-27T00:00:00Z',
  'expiration_time': '2024-04-02T23:00:00Z',
  'latest_expiration_time': '2024-04-02T23:00:00Z',
  'settlement_timer_seconds': 3600,
  'status': 'determined',
  'response_price_units': 'usd_cent',
  'notional_value': 100,
  'tick_size': 1,
  'yes_bid': 99,
  'yes_ask': 100,
  'no_bid': 0,
  'no_ask': 0,
  'last_price': 0,
  'previous_yes_bid': 0,
  'previous_yes_ask': 100,
  'previous_price': 0,
  'volume': 0,
  'volume_24h': 0,
  'liquidity': 0,
  'open_interest': 0,
  'result': 'yes',
  'settlement_value': 100,
  'can_close_early': True,
  'expiration_value': '5203.58',
  'category': '',
  'risk_limit_cents': 0,
  'strike_type': 'greater',
  '

In [32]:
#Kalshi SP500 above/below market goes back to Jan 19, 2024

'''
get bid and ask quotes for SPX above/below markets on Kalshi
'''
lb = None

ndx_df, spx_df = get_ndx_spx_data()
    
ndx_df = ndx_df[ndx_df['Date'] >= date(2024, 1, 1)][['Date', 'Open', 'Close']].reset_index(drop=True)
spx_df = spx_df[spx_df['Date'] >= date(2024, 1, 1)][['Date', 'Open', 'Close']].reset_index(drop=True) #get daily index data for 2024 onwards


spx_d = {'bid': [], 'ask': [], 'kalshi_strike': [], 'underlying': [], 'datetime': [], 'date': []}
for idx, row in spx_df.iterrows():
    dt = row['Date']
    if dt < date(2024, 1, 19):
        continue
    spx_close = float(row['Close'])
    event_ticker = 'INXU-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        print(event_ticker)
        continue
    market_tickers = []
    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 9, 0, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        lb = float(split[0].replace(',',''))
        
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 9:
            bb = info['yes_bid']
            ba = info['yes_ask']
            spx_d['bid'].append(bb)
            spx_d['ask'].append(ba)
            spx_d['kalshi_strike'].append(lb)
            spx_d['datetime'].append(first_ts)
            spx_d['date'].append(first_ts.date())
            spx_d['underlying'].append('SPX')
        
spx_d = pd.DataFrame(data=spx_d)

In [34]:
spx_d[spx_d['date']==date(2024, 3, 26)]

,bid,ask,kalshi_strike,underlying,datetime,date
401,97,100,5175.0,SPX,2024-03-26 09:00:01,2024-03-26
402,92,94,5200.0,SPX,2024-03-26 09:02:10,2024-03-26
403,74,77,5218.2,SPX,2024-03-26 09:00:03,2024-03-26
404,63,67,5225.0,SPX,2024-03-26 09:00:00,2024-03-26
405,19,22,5250.0,SPX,2024-03-26 09:00:36,2024-03-26
406,0,4,5275.0,SPX,2024-03-26 09:14:48,2024-03-26


In [36]:
'''
get bid/ask quotes for NDX above/below markets on Kalshi
'''

from KalshiClientsBaseV2 import HttpError
lb = None

ndx_d = {'bid': [], 'ask': [], 'kalshi_strike': [], 'underlying': [], 'datetime': [], 'date': []}
for idx, row in ndx_df.iterrows():
    dt = row['Date']
    if dt < date(2024, 1, 19):
        continue
    event_ticker = 'NASDAQ100U-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        event_ticker += '0' + str(dt.day)
    else:
        event_ticker += str(dt.day)
    try:
        event = exchange_client.get_event(event_ticker)
    except:
        continue

    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 9, 30, 0).timetuple()))
    for m in event['markets']:
        ticker = m['ticker']
        subtitle = m['yes_sub_title']
        split = subtitle.split(' ')
        lb = float(split[0].replace(',',''))
    
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except HttpError:
            continue

        info = r['history'][0]
        first_ts = datetime.fromtimestamp(info['ts'])
        if first_ts.hour == 9:
            bb = info['yes_bid']
            ba = info['yes_ask']
            ndx_d['bid'].append(bb)
            ndx_d['ask'].append(ba)
            ndx_d['kalshi_strike'].append(lb)
            ndx_d['underlying'].append('NDX')
            ndx_d['datetime'].append(first_ts)
            ndx_d['date'].append(first_ts.date())
         
ndx_d = pd.DataFrame(data=ndx_d)

In [37]:
ndx_d

,bid,ask,kalshi_strike,underlying,datetime,date
0,99,100,16600.00,NDX,2024-01-19 09:46:10,2024-01-19
1,99,100,16700.00,NDX,2024-01-19 09:46:06,2024-01-19
2,94,100,16800.00,NDX,2024-01-19 09:30:00,2024-01-19
3,89,99,16900.00,NDX,2024-01-19 09:30:00,2024-01-19
4,0,100,17000.00,NDX,2024-01-19 09:30:00,2024-01-19
...,...,...,...,...,...,...
462,69,75,18277.07,NDX,2024-03-26 09:30:00,2024-03-26
463,63,68,18300.00,NDX,2024-03-26 09:30:00,2024-03-26
464,28,33,18400.00,NDX,2024-03-26 09:30:00,2024-03-26
465,4,7,18500.00,NDX,2024-03-26 09:30:00,2024-03-26


In [38]:
ndx_d[ndx_d['date']==date(2024, 3, 26)]

,bid,ask,kalshi_strike,underlying,datetime,date
457,98,100,17800.00,NDX,2024-03-26 09:30:15,2024-03-26
458,99,100,17900.00,NDX,2024-03-26 09:31:00,2024-03-26
459,99,100,18000.00,NDX,2024-03-26 09:31:00,2024-03-26
460,96,98,18100.00,NDX,2024-03-26 09:30:03,2024-03-26
461,87,91,18200.00,NDX,2024-03-26 09:30:00,2024-03-26
462,69,75,18277.07,NDX,2024-03-26 09:30:00,2024-03-26
463,63,68,18300.00,NDX,2024-03-26 09:30:00,2024-03-26
464,28,33,18400.00,NDX,2024-03-26 09:30:00,2024-03-26
465,4,7,18500.00,NDX,2024-03-26 09:30:00,2024-03-26
466,0,1,18700.00,NDX,2024-03-26 09:31:00,2024-03-26


In [39]:
kalshi_df = pd.concat([spx_d, ndx_d])

In [40]:
df_otm.columns

Index(['underlying', 'strike', 'type', 'date', 'price', 'Open', 'Close',
       'est_iv', 'geq_strike_prob'],
      dtype='object')

In [41]:
kalshi_df.columns

Index(['bid', 'ask', 'kalshi_strike', 'underlying', 'datetime', 'date'], dtype='object')

In [42]:
df_otm = df_otm.drop(columns=['est_iv'])
kalshi_df['strike'] = kalshi_df['kalshi_strike'].to_list()

In [43]:
kalshi_df = kalshi_df.drop(columns=['kalshi_strike'])

In [ ]:
res = df_otm.merge(kalshi_df, on=['underlying', 'date', 'strike'])